<a href="https://colab.research.google.com/github/DeependraChaddha/RAG_Projects/blob/main/RAG_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setting up Environment

Importing Packages

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

Langsmith


In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOINT']='https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']= ######FILL API KEY HERE######

API Key

In [ ]:
os.environ['OPENAI_API_KEY']=######FILL API KEY HERE######

##Overview


make imports

In [ ]:
import bs4 #BeautifulSoup for extracting data from HTML or XML files
from langchain import hub #tool for managing components from LangChain framework
from langchain.text_splitter import RecursiveCharacterTextSplitter #for text splitting
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


INDEXING

In [ ]:
#Loading documents
loader=WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
        class_=("post-content", "post-title", "post-header")
                )
        ),
)#Made an instance of the WebBaseLoader class from langchain_community package
docs=loader.load()#used load method of this class

#Split
text_splitter=RecursiveCharacterTextSplitter(chunksize=1000, chunk_overlap=200)
splits=text_splitter.split_documents(docs)

#Embed
vectorstore=Chroma.from_documents(documents=splits,
                                  embedding=OpenAIEmbeddings())
retriever=vectorstore.as_retriever()


RETRIEVAL AND GENERATION

In [ ]:
#prompt
prompt=hub.pull("rlm/rag-prompt")

#LLM
llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

#post-processing
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

#Chain
rag_chain= (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#Question
rag_chain.invoke("Any Question?")

##Indexing

In [ ]:
#Documents
question="Any Question?"
document="any answer/document"

In [ ]:
import tiktoken

def num_tokens_from_string(string:str, encoding_name:str) -> int: ###This function returns the number of tokens in a string
  encoding=tiktoken.get_encoding(encoding_name)
  num_tokens=len(encoding.encode(string))
  return num_tokens

num_token_from_string(question, "cl100kbase") #takes string to convert to tokens and string which tells the name of encoding being used
